In [1]:
import psycopg2 as pg
import psycopg2.extras as pgExtras
import json
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

In [2]:
params = {  'user': 'cristiano', 'password': 'cristiano',
            'host':'localhost', 'port':'5432', 'database':'afterqualifying'}

def cleanTable():
    conn = None
    try:
        conn = pg.connect(**params)
        cur = conn.cursor()
        
        cur.execute('delete from PLACE_HEATMAP')
        
        conn.commit()
        cur.close()

        print('Heatmap table cleaned.')

    except(Exception, pg.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

def updatePlaceHeatmapStations():
    conn = None
    try:
        conn = pg.connect(**params)
        cur = conn.cursor()
        
        cur.execute(''' update	PLACE_HEATMAP

                        set		GEOM = ST_LineInterpolatePoint(EDGE.GEOM, STATION.POSITIONINEDGE),
                                IDEDGE_FK = EDGE.IDEDGE,
                                POSITIONINEDGE = STATION.POSITIONINEDGE

                        from	STREETSEGMENT EDGE, STATION

                        where	EDGE.IDEDGE = STATION.IDEDGE_FK and
                                STATION.IDSTATION = PLACE_HEATMAP.IDPLACE and
                                PLACE_HEATMAP.TYPEOFPLACE = 'STATION'   ''')

        conn.commit()
        cur.close()

        print('Heatmap Stations updated.')
    
    except(Exception, pg.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
    
def updatePlaceHeatmapTrips():
    conn = None
    try:
        conn = pg.connect(**params)
        cur = conn.cursor()
        
        cur.execute(''' update	PLACE_HEATMAP

                        set		GEOM = PLACE.GEOM,
                                IDEDGE_FK = PLACE.IDEDGE_FK,
                                POSITIONINEDGE = PLACE.POSITIONINEDGE

                        from	PLACE

                        where	PLACE_HEATMAP.IDPLACE = PLACE.IDPLACE and
                                PLACE_HEATMAP.TYPEOFPLACE = 'TRIP'   ''')

        conn.commit()
        cur.close()
        
        print('Heatmap Trips updated.')
    
    except(Exception, pg.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [3]:
def reopenConn(conn, cur, params):
    conn.commit()
    cur.close()
    conn.close()
    conn = pg.connect(**params)
    cur = conn.cursor()

    return conn, cur

class Place:
    currentDescription = None

    def __init__(self, idPlace, edgeId, positionInEdge, valueHeatMap):
        self.idPlace = idPlace
        self.edgeId = edgeId
        self.positionInEdge = positionInEdge
        self.valueHeatMap = valueHeatMap

def loadStationsNumberVehiclesParkingSlots(fileName):
    places = list()

    with open(fileName) as jsonFile:
        optimalSolution = json.load(jsonFile)

        for var in optimalSolution["Vars"]:
            varVTag = var["VTag"][0]
            varVTagSplit = varVTag.split('_')

            if varVTag.startswith('station') and varVTag.endswith('start'):
                amountVehicles = int(var["X"])
                if amountVehicles > 0:
                    idStation = int(varVTagSplit[1])
                    place = Place(idStation, None, None, amountVehicles)
                    places.append(place)

    return places

def storeStations(fileName, description):
    Place.currentDescription = description
    places = loadStationsNumberVehiclesParkingSlots(fileName)
    
    conn = None
    try:
        conn = pg.connect(**params)
        cur = conn.cursor()
                
        lengthToInsert = 100000
        placesToInsert = list()
        sqlInsert = 'insert into PLACE_HEATMAP (IDPLACE, DESCRIPTION, VALUEHEATMAP, TYPEOFPLACE) values %s'
        for place in places:
            placesToInsert.append((place.idPlace, 'Initial Vehicles ' + Place.currentDescription, place.valueHeatMap, 'STATION'))
            
            if len(placesToInsert) > lengthToInsert:
                pgExtras.execute_values(cur, sqlInsert, placesToInsert)
                placesToInsert = []
                conn, cur = reopenConn(conn, cur, params)
        
        if len(placesToInsert) > 0:
            pgExtras.execute_values(cur, sqlInsert, placesToInsert)

        conn.commit()
        cur.close()

        print(description + ' inserted into Heatmap.')

    except(Exception, pg.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()


In [4]:
class Trip:
    #Values for defining the new timestamps
    ADJUSTED_MONTH = 1
    ADJUSTED_YEAR = 2017

    def adjustDay(timeDepartureOld, drivingDuration):
        #The weekday() starts from 0 but it does not exist a day 0 in the calendar. Then, a +1 solves this issue
        adjustedDayDeparture = timeDepartureOld.weekday() + 1

        timeDepartureNew = timeDepartureOld.replace(day=adjustedDayDeparture, month=Trip.ADJUSTED_MONTH, year=Trip.ADJUSTED_YEAR)
        timeArrivalNew = timeDepartureNew + timedelta(minutes=drivingDuration)
        
        return timeDepartureNew, timeArrivalNew

    def __init__(self, idTrip, expansionFactor, placeStart, placeEnd, timestampDeparture, timestampArrival, drivingDistance, drivingDuration):
        self.idTrip = idTrip
        self.expansionFactor = expansionFactor
        self.placeStart = placeStart
        self.placeEnd = placeEnd
        self.timestampDepartureOld = timestampDeparture
        self.timestampArrivalOld = timestampArrival
        self.drivingDistance = drivingDistance
        self.drivingDuration = drivingDuration

        self.timestampDeparture, self.timestampArrival = Trip.adjustDay(timestampDeparture, self.drivingDuration)

def idPlaceToInt(idPlace):
    intIdPlace = int(idPlace.replace('place_', ''))

    return intIdPlace

def loadTrips():
    conn = pg.connect(**params)

    sqlQuery = '''	select	'trip_'||TRIP.IDTRIP::text as IDTRIP,
                            TRIP.TRIPEXPANSIONFACTOR,
                            'place_'||TRIP.IDPLACEDEPARTURE::text as IDPLACEDEPARTURE,
                            'place_'||TRIP.IDPLACEDESTINATION::text as IDPLACEDESTINATION,
                            TRIP.TIMESTAMPDEPARTURE,
                            TRIP.TIMESTAMPARRIVAL,
                            TRIP.DRIVINGDISTANCE,
                            TRIP.DRIVINGDURATION
                    from	TRIP
                    where   TRIP.DRIVINGDISTANCE > 500
                    '''
    dataFrameEdges = pd.read_sql_query(sqlQuery, conn)
    conn.close()

    trips = dict()
    for row in dataFrameEdges.itertuples():
        dictRow = row._asdict()
        trip = Trip(idTrip=dictRow['idtrip'],
                    expansionFactor=dictRow['tripexpansionfactor'],
                    placeStart=idPlaceToInt(dictRow['idplacedeparture']),
                    placeEnd=idPlaceToInt(dictRow['idplacedestination']),
                    timestampDeparture=dictRow['timestampdeparture'],
                    timestampArrival=dictRow['timestamparrival'],
                    drivingDistance=dictRow['drivingdistance'],
                    drivingDuration=dictRow['drivingduration'])
        trips[trip.idTrip] = trip

    return trips

def loadTripsServedStartEnd(fileName, tripsDatabase):
    tripsStart = list()
    tripsEnd = list()

    with open(fileName) as jsonFile:
        optimalSolution = json.load(jsonFile)

        for var in optimalSolution["Vars"]:
            varVTag = var["VTag"][0]
            varVTagSplit = varVTag.split('_')

            if varVTag.startswith('trip'):
                idTrip = 'trip_' + varVTagSplit[1]
                amountServed = int(var["X"])
                if amountServed > 0:
                    trip = tripsDatabase[idTrip]
                    trip.valueHeatMap = amountServed
                    if varVTagSplit[2] == 'start':
                        tripsStart.append(trip)
                    elif varVTagSplit[2] == 'end':
                        tripsEnd.append(trip)
                    
    return tripsStart, tripsEnd

def storeTrips(fileName, description, tripsDatabase):
    Place.currentDescription = description
    tripsStart, tripsEnd = loadTripsServedStartEnd(fileName, tripsDatabase)
    
    conn = None
    try:
        conn = pg.connect(**params)
        cur = conn.cursor()
                
        lengthToInsert = 100000
        placesToInsert = list()
        sqlInsert = 'insert into PLACE_HEATMAP (IDPLACE, DESCRIPTION, VALUEHEATMAP, TYPEOFPLACE) values %s'
        for trip in tripsStart:
            placesToInsert.append((trip.placeStart, 'Start ' + Place.currentDescription, trip.valueHeatMap, 'TRIP'))
        
        for trip in tripsEnd:
            placesToInsert.append((trip.placeEnd, 'End ' + Place.currentDescription, trip.valueHeatMap, 'TRIP'))

            if len(placesToInsert) > lengthToInsert:
                pgExtras.execute_values(cur, sqlInsert, placesToInsert)
                placesToInsert = []
                conn, cur = reopenConn(conn, cur, params)
        
        if len(placesToInsert) > 0:
            pgExtras.execute_values(cur, sqlInsert, placesToInsert)

        conn.commit()
        cur.close()

        print(description + ' inserted into Heatmap.')
    
    except(Exception, pg.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [5]:
def loadBalanceData(fileName, tripsDatabase):
    stationsStart = dict()
    stationsBalance = list()
    allocatedStations = 0
    with open(fileName) as jsonFile:
        optimalSolution = json.load(jsonFile)

        for var in optimalSolution["Vars"]:
            varVTag = var["VTag"][0]
            varVTagSplit = varVTag.split('_')

            if varVTag.startswith('station') and varVTag.endswith('start'):
                amountVehicles = int(var["X"])
                if amountVehicles > 0:
                    allocatedStations += amountVehicles

                    idStation = int(varVTagSplit[1])
                    stationsStart[idStation] = amountVehicles
                    #stationsBalance.append((idStation, INITIAL_TIMESTAMP, amountVehicles))

            elif varVTag.startswith('trip'):
                idTrip = 'trip_' + varVTagSplit[1]
                if varVTagSplit[2] in ['start', 'end']:
                    idStation = int(varVTagSplit[4])
                    vehiclesFlow = int(var["X"])
                    if varVTagSplit[2] == 'start':
                        stationsBalance.append((idStation, tripsDatabase[idTrip].timestampDeparture, -1*vehiclesFlow))
                    else:
                        stationsBalance.append((idStation, tripsDatabase[idTrip].timestampArrival, vehiclesFlow))
    
    return stationsBalance, stationsStart

def calculateSurpluses(fileName, tripsDatabase):
    surpluses, stationsStart = loadBalanceData(fileName, tripsDatabase)

    surpluses.sort()
    i = 0
    while i < len(surpluses):
        currentStation = surpluses[i][0]
        if currentStation not in stationsStart:
            stationsStart[currentStation] = 0

        earlierBalance = stationsStart[currentStation]

        while i < len(surpluses) and currentStation == surpluses[i][0]:
            #Initially, idStation comes first in the tuple sequence to keep the sorting rationale
            (idStation, timestamp, vehiclesFlow) = surpluses[i]
            surpluses[i] = (timestamp, idStation, earlierBalance - stationsStart[currentStation], earlierBalance)
            earlierBalance += vehiclesFlow

            i += 1
    #Repeating this line because there is no do_while loop in Python
    surpluses[i-1] = (timestamp, idStation, earlierBalance - stationsStart[currentStation], earlierBalance)

    return surpluses

def mergeExtremePoints(dataFrame, minOrMax):
    #Fixing pandas weekday to start at Sunday, as matplotlib and datetime uses
    dataFrame['weekday'] = dataFrame['timestamp'].dt.weekday
    dataFrame['weekday'] = np.where(dataFrame['weekday'] <= 5, dataFrame['weekday'] + 1, 0)

    if minOrMax == 'min':
        points = dataFrame.groupby(dataFrame['weekday'])[['calcbalance']].min()
    elif minOrMax == 'max':
        points = dataFrame.groupby(dataFrame['weekday'])[['calcbalance']].max()

    points.reset_index(inplace=True)
    points = pd.merge(points, dataFrame, on=['weekday', 'calcbalance'])
    points = points.groupby(points['weekday']).first()

    return points

def buildSurplusesDataFrames(surpluses, weekDay):
    dataFramePd = pd.DataFrame(surpluses, columns=['timestamp', 'station', 'calcbalance', 'balance'])
    dataFramePd = dataFramePd.set_index('timestamp')
    dataFramePd['timestamp'] = dataFramePd.index
    #print(dataFramePd.describe())

    #Adjust the last points to the beginning of the first week
    INITIAL_TIMESTAMP = datetime(year=Trip.ADJUSTED_YEAR, month=Trip.ADJUSTED_MONTH, day=1, hour=0, minute=0, second=0)
    mask = dataFramePd['timestamp'] >= INITIAL_TIMESTAMP + timedelta(weeks=1)
    dataFramePd.loc[mask, 'timestamp'] = dataFramePd.loc[mask, 'timestamp'].apply(lambda dateTooLate: dateTooLate - timedelta(weeks=1))

    #dataFramePd['weekday'] = dataFramePd['timestamp'].dt.weekday
    #dataFramePd = dataFramePd.loc[dataFramePd['weekday'] == weekDay]

    #minPoints = mergeExtremePoints(dataFrame=dataFramePd, minOrMax='min')
    maxPoints = mergeExtremePoints(dataFrame=dataFramePd, minOrMax='max')
    #redPoints = pd.concat([minPoints, maxPoints])

    #Selecting only common surpluses about the time when the maximum happened
    timestampMaxWeekDay = maxPoints.loc[weekDay, 'timestamp']
    dataFramePd = dataFramePd.loc[dataFramePd['timestamp'] <= timestampMaxWeekDay]
    groupByDF = dataFramePd.groupby(dataFramePd['station'])[['timestamp']].max()
    dataFramePd.reset_index(inplace=True, drop=True)
    dataFramePd = pd.merge(dataFramePd, groupByDF, on=['station', 'timestamp'])

    return dataFramePd#, redPoints

def storeMidnightSurpluses(description, surpluses, weekDay):
    Place.currentDescription = description
    dataFramePd = buildSurplusesDataFrames(surpluses=surpluses, weekDay=weekDay)
    #Useful for using the strftime to get the weekday name
    fooTimestampWeekDay = datetime(year=Trip.ADJUSTED_YEAR, month=Trip.ADJUSTED_MONTH, day=weekDay + 1, hour=0, minute=0, second=0)
    conn = None
    try:
        conn = pg.connect(**params)
        cur = conn.cursor()

        lengthToInsert = 100000
        placesToInsert = list()
        sqlInsert = 'insert into PLACE_HEATMAP (IDPLACE, DESCRIPTION, VALUEHEATMAP, TYPEOFPLACE) values %s'

        for index, row in dataFramePd.iterrows():
            placesToInsert.append((row['station'], 'Surplus ' + fooTimestampWeekDay.strftime('%A') + ' ' + Place.currentDescription, row['calcbalance'], 'STATION'))

            if len(placesToInsert) > lengthToInsert:
                pgExtras.execute_values(cur, sqlInsert, placesToInsert)
                placesToInsert = []
                conn, cur = reopenConn(conn, cur, params)
        
        if len(placesToInsert) > 0:
            pgExtras.execute_values(cur, sqlInsert, placesToInsert)

        conn.commit()
        cur.close()

        print('Surpluses ' + fooTimestampWeekDay.strftime('%A') + ' ' + description + ' inserted into Heatmap.')
    
    except(Exception, pg.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [6]:
cleanTable()

folder = 'Optimal Solutions/'
filenamesDescriptions = [   (folder + 'Free Floating/4000/500/70.json', 'Free Floating 4000 km 500 m 0.7'),
                            (folder + 'Free Floating/4000/500/200.json', 'Free Floating 4000 km 500 m 2'),
                            (folder + 'Free Floating/3000/500/70.json', 'Free Floating 3000 km 500 m 0.7'),
                            (folder + 'Mixed Free Floating/3000/500/90.json', 'Mixed Free Floating 3000 km 500 m 0.9'),
                            (folder + 'Restricted 1 31 Partial Floating/3000/500/90.json', 'Restricted 1 31 Partial Floating 3000 km 500 m 0.9'),
                            (folder + 'Restricted 1 62 Partial Floating/3000/500/90.json', 'Restricted 1 62 Partial Floating 3000 km 500 m 0.9'),
                            (folder + 'Restricted 1 inf Partial Floating/3000/500/90.json', 'Restricted 1 inf Partial Floating 3000 km 500 m 0.9')
                        ]

for filename, description in filenamesDescriptions:
    storeStations(  fileName=filename,
                    description=description)

tripsDatabase = loadTrips()
for filename, description in filenamesDescriptions:
    storeTrips( fileName=filename,
                description=description,
                tripsDatabase=tripsDatabase)

for filename, description in filenamesDescriptions:
    surpluses = calculateSurpluses( fileName=filename,
                                    tripsDatabase=tripsDatabase)
    for weekDay in range(7):
        storeMidnightSurpluses( description=description,
                                surpluses=surpluses,
                                weekDay=weekDay)

updatePlaceHeatmapStations()
updatePlaceHeatmapTrips()

Heatmap table cleaned.
Free Floating 4000 km 500 m 0.7 inserted into Heatmap.
Free Floating 4000 km 500 m 2 inserted into Heatmap.
Free Floating 3000 km 500 m 0.7 inserted into Heatmap.
Mixed Free Floating 3000 km 500 m 0.9 inserted into Heatmap.
Restricted 1 31 Partial Floating 3000 km 500 m 0.9 inserted into Heatmap.
Restricted 1 62 Partial Floating 3000 km 500 m 0.9 inserted into Heatmap.
Restricted 1 inf Partial Floating 3000 km 500 m 0.9 inserted into Heatmap.
Free Floating 4000 km 500 m 0.7 inserted into Heatmap.
Free Floating 4000 km 500 m 2 inserted into Heatmap.
Free Floating 3000 km 500 m 0.7 inserted into Heatmap.
Mixed Free Floating 3000 km 500 m 0.9 inserted into Heatmap.
Restricted 1 31 Partial Floating 3000 km 500 m 0.9 inserted into Heatmap.
Restricted 1 62 Partial Floating 3000 km 500 m 0.9 inserted into Heatmap.
Restricted 1 inf Partial Floating 3000 km 500 m 0.9 inserted into Heatmap.
Surpluses Sunday Free Floating 4000 km 500 m 0.7 inserted into Heatmap.
Surpluses M